# Bioeconomy visions relation to eco-innovations

For this I need:

1. the eco-innovations associated with no bioeconomy vision [x]
2. the number of bio-economy visions with no eco-innovations [x]
3. the total number of bioeconomy visions [x]
4. the total number of eco-innovations [x]

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from src.swinno_helpers import connect_swinno_db
from src.utils import get_project_root

ROOT = get_project_root()
conn = connect_swinno_db()


## Eco-Innovations with no bioeconomy vision

In [4]:
ei_no_bv = pd.read_sql(
    """
SELECT
  ei.sinno_id,
  ei.innovation_type
FROM
  eco_innovations_articles AS ei
  INNER JOIN bioeconomy_visions_articles AS bv ON ei.sinno_id = bv.sinno_id
WHERE
  bv.bioeconomy_vision = 9
  AND bv.sinno_id NOT IN (
    SELECT
      sinno_id
    FROM
      bioeconomy_visions_articles AS bv2
    WHERE
      bv2.bioeconomy_vision = 0
  )
  AND ei.innovation_type <> 999;
""",
    conn,
)


## Bioeconomy vision with no eco-innovations

In [57]:
bv_no_ei = pd.read_sql(
    """
SELECT
  DISTINCT(bv.sinno_id)
FROM
  bioeconomy_visions_articles as bv
  JOIN eco_innovations_articles as ei on bv.sinno_id = ei.sinno_id
WHERE
  bv.sinno_id <> 9
  AND bv.sinno_id NOT IN(
    SELECT
      sinno_id
    FROM
      bioeconomy_visions_articles AS bv2
    WHERE
      bv2.bioeconomy_vision = 0
  )
  AND bv.sinno_id NOT IN (
    SELECT
      sinno_id
    FROM
      eco_innovations_articles AS ei2
    WHERE
      ei2.innovation_type = 999
  );
""",
    conn,
)

len(bv_no_ei)


54

## Total Numbers

Should this be on a per innovation basis or counted on the number of tags?
Start with the number of tags.

Needs to be the number of unique innovations.

In [58]:
bvs = pd.read_sql(
    """
SELECT
  bv.sinno_id,
  bv.bioeconomy_vision
FROM
  bioeconomy_visions_articles as bv
WHERE 
  bv.sinno_id <> 9
  AND bv.sinno_id NOT IN(
    SELECT
      sinno_id
    FROM
      bioeconomy_visions_articles AS bv2
    WHERE
      bv2.bioeconomy_vision = 0
  );
""",
    conn,
)  # filter out unsures
len(bvs)


91

In [59]:
eis = pd.read_sql(
    """
SELECT
    sinno_id
FROM 
    eco_innovations_articles as ei
WHERE
    ei.sinno_id NOT IN (
    SELECT
      sinno_id
    FROM
      eco_innovations_articles AS ei2
    WHERE
      ei2.innovation_type = 0 OR ei2.innovation_type = 000
  );
""",
    conn,
)  # filter out unsures


#### Shares

In [60]:
bv_share = len(bv_no_ei) / len(bvs)
bv_share_unique = len(bv_no_ei["sinno_id"].unique()) / len(bvs["sinno_id"].unique())

ei_share = len(ei_no_bv) / len(eis)
ei_share_unique = len(ei_no_bv["sinno_id"].unique()) / len(eis["sinno_id"].unique())

print(f"Share of articles with BV but no EI: {bv_share:.2%}")
print(f"Share of articles with BV but no EI (unique): {bv_share_unique:.2%}")
print(f"Share of articles with EI but no BV: {ei_share:.2%}")
print(f"Share of articles with EI but no BV (unique): {ei_share_unique:.2%}")


Share of articles with BV but no EI: 59.34%
Share of articles with BV but no EI (unique): 61.36%
Share of articles with EI but no BV: 19.76%
Share of articles with EI but no BV (unique): 19.64%


These results make no sense for the bioeconomy visions, as the share is way to high